## Text Feature Extractor
this notebook generates for each text some additional/handcrafted features characterizing its named entity composition and their relative relative position (entity distance):distance is the number of words separating 2 entities (it can be negative)
* number of sentences
* number of words
* distinct count of drug name entities
* distinct count of active ingredient entities
* count of question marks (typically to identify specifically multi-intent label)
* individual count of interrogative pronoun entities (one column per pronoun: quand, qui, quoi, ou, comment, pourquoi, combien, quel(s|le,..)
* distinct count of time entities (eg: jours, après midi, soir, année, 12h, mardi, samedi, temps....)
* distinct count of quantity entities (eg: 5mg, 10ml, ...)
* count of association entities (eg: et, avec, ou, ...)
* distance between interrogative pronoun and drug name entities
* distance between active ingredient and drug name entities
* distance between quantity and drug name entities
* distance between time and drug name entities
* distance between question marks and drug name entities

In addition, we compute the likelihood of the document to be part of the **topics identified earlier by NMF **:
*  this probability is basically the ratio of matching top word over the total count

eg: topic1 is characterized by 10 top words (A, B, C, D, ....)

if the text contains words B and E, the  topic likelihood would be 2 / 10 = 1 /5

these extracted features are stored into this [file](../../data/staging_data/text_extracted_features.csv) with ID column to join it with the former train dataset if required.

In [1]:
import pandas as pd
import numpy as np

XTrain = pd.read_csv('../../data/staging_data/mispelling_fixed_clean_input_train.csv', sep=',')

In [2]:
from collections import Counter
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def words(text):
    return re.findall(r'\w+', text.lower())

# value domain based named entities
drugNames = set(words(open('../../data/staging_data/drug_names.txt').read()))
ingredientNames = set(words(open('../../data/staging_data/ingredient_names.txt').read()))

# reg expression based named entities
questionMarkRegEx = r"[\?]+"
compiledquestionMarkRegEx = re.compile(questionMarkRegEx)

sentenceSeparatorRegEx = r"[\?]+|!|:|;|[\.]+"
#quiRegEx = r"[\s\?\.\!\:\;]?[Q|q]ui\s" qui term is ambiguous
combienRegEx = r"[\s\?\.\!\:\;]?[C|c]ombien\s"
pourquoiRegEx = r"[\s\?\.\!\:\;]?[P|p]ourquoi\s"
quandRegEx = r"[\s\?\.\!\:\;]?[Q|q]uand\s"
quoiRegEx = r"[\s\?\.\!\:\;]?[Q|q]uoi\s|[\s\?\.\!\:\;]?[Q|q]uel[l|s|le|les]?\s"
commentRegEx = r"[\s\?\.\!\:\;]?[C|c]omment\s"
interrogativePronounRegEx = combienRegEx + '|' + pourquoiRegEx + '|' + quandRegEx + '|' + quoiRegEx + '|' + commentRegEx
compiledPronounRegEx = re.compile(interrogativePronounRegEx)

# associative entity
ouRegEx = r"\sou\s"
etRegEx = r"\set\s"
avecRegEx = r"\savec\s"


# gold standard
timeRegEx = r"([D|d]emain|[H|h]ier|[M|m]atin|[M|m]idi|[S|s]oir|mois|jours|[D|d]epuis|semaine[s|S]|heure[s|S])( )*( |,|\.)"
quantityRegEx = r"[0-9]+\s?[m|k]?g|\s[m|k]?g[^a-zA-Zéàèî]|[0-9]+\s?[m|c]?l|\s[m|c]?l[^a-zA-Zéàèî]"

def getWordCount(text):
    words = word_tokenize(text, language='french')
    return len(words)

def getEntityCount(text, entityValueSet, lowerCase):
    ''' return distinct count of entity occurrences found from the text'''
    words = word_tokenize(text, language='french')
    if lowerCase:
        words = map(lambda x: x.lower(), words)
    return len(list(set(words) & entityValueSet))

def countWordsBefore(text, lastPosition):
    ''' count number of words before this position'''
    text = text[0:lastPosition]
    words = word_tokenize(text, language='french')
    return len(words)

def getWordIndexByReg(text, regEx):
    ''' return the word index of the first group matching the regexp'''
    iterables = list(regEx.finditer(text))
    positions = list(map(lambda x : x.span()[0], iterables))
    if len(positions) > 0:
        position = int(positions[0])
        return countWordsBefore(text, position)
    else:
        return None

def getWordIndexByValueSet(text, valueSet):
    ''' return the word index of the first token part of the value domain'''
    words = word_tokenize(text, language='french')
    wordIndex = 0
    for word in words:
        if word in valueSet:
            return wordIndex
        wordIndex = wordIndex + 1
    return None

[nltk_data] Downloading package punkt to /Users/i051796/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
XTrain['drugCount'] = XTrain['question'].map(lambda text: getEntityCount(text, drugNames, True))
XTrain['ingredientCount'] = XTrain['question'].map(lambda text: getEntityCount(text, ingredientNames, True))

In [4]:
XTrain['timeCount'] = XTrain['question'].map(lambda text : len(re.findall(timeRegEx, text)))
XTrain['quantitiesCount'] = XTrain['question'].map(lambda text : len(re.findall(quantityRegEx, text)))
XTrain['questionMarkCount'] = XTrain['question'].map(lambda text : len(re.findall(questionMarkRegEx, text)))
XTrain['sentenceCount'] = XTrain['question'].map(lambda text : 1 + len(re.findall(sentenceSeparatorRegEx, text)))
XTrain['wordCount'] = XTrain['question'].map(lambda text : getWordCount(text))

In [5]:
XTrain['combienCount'] = XTrain['question'].map(lambda text : len(re.findall(combienRegEx, text)))
XTrain['pourquoiCount'] = XTrain['question'].map(lambda text : len(re.findall(pourquoiRegEx, text)))
XTrain['quandCount'] = XTrain['question'].map(lambda text : len(re.findall(quandRegEx, text)))
XTrain['quoiCount'] = XTrain['question'].map(lambda text : len(re.findall(quoiRegEx, text)))
XTrain['commentCount'] = XTrain['question'].map(lambda text : len(re.findall(commentRegEx, text)))

In [6]:
XTrain['avecCount'] = XTrain['question'].map(lambda text : len(re.findall(avecRegEx, text)))
XTrain['etCount'] = XTrain['question'].map(lambda text : len(re.findall(etRegEx, text)))
XTrain['ouCount'] = XTrain['question'].map(lambda text : len(re.findall(ouRegEx, text)))

In [7]:
XTrain['interrogative_WordIndex'] = XTrain['question'].map(lambda text : getWordIndexByReg(text, compiledPronounRegEx))
XTrain['questionMark_WordIndex'] = XTrain['question'].map(lambda text : getWordIndexByReg(text, compiledquestionMarkRegEx))

In [8]:
XTrain['drug_WordIndex'] = XTrain['question'].map(lambda text : getWordIndexByValueSet(text, drugNames))
XTrain['drug_InterrogativeDistance'] = XTrain['interrogative_WordIndex'] - XTrain['drug_WordIndex']
XTrain['drug_QuestionMarkDistance'] = XTrain['questionMark_WordIndex'] - XTrain['drug_WordIndex']

In [9]:
pd.set_option("display.max_colwidth",1000)
XTrain.drop(["question"], axis=1).head(5)

,ID,drugCount,ingredientCount,timeCount,quantitiesCount,questionMarkCount,sentenceCount,wordCount,combienCount,pourquoiCount,...,quoiCount,commentCount,avecCount,etCount,ouCount,interrogative_WordIndex,questionMark_WordIndex,drug_WordIndex,drug_InterrogativeDistance,drug_QuestionMarkDistance
0,0,0,1,3,0,0,1,74,0,0,...,0,0,0,3,0,NaN,None,NaN,NaN,None
1,1,1,0,0,0,0,1,10,0,0,...,0,0,0,0,0,NaN,None,4.0,NaN,None
2,2,1,0,0,0,0,1,50,0,0,...,0,0,0,1,0,NaN,None,4.0,NaN,None
3,3,1,0,0,0,0,1,14,0,0,...,0,0,0,0,0,NaN,None,13.0,NaN,None
4,4,1,1,0,0,0,1,22,0,0,...,0,0,0,1,0,NaN,None,12.0,NaN,None


### topic likelihood features

In [10]:
topicTopWords = pd.read_csv('../../data/staging_data/topic_top_words.csv')
nbTopWords = len(topicTopWords.columns) - 1
nbTopics = len(topicTopWords)
# build a dictionary with topic id as key and bag of top words as value
dictOfBagTopWords = dict()

for row in topicTopWords.iterrows():    
    topic_id = row[1]['topic_id']
    values = row[1].values
    dictOfBagTopWords[values[0]] = set (values[1:])

In [11]:
import sys
sys.path.append('..')
from utils import textcleaner
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

frenchStopWords = set(stopwords.words("french"))

topicLikelihoods = []

for text in XTrain.question.values:        
    bagOfWords = set(textcleaner.cleanText(text, frenchStopWords).split())
    topiclikelihoodRow = [None] * nbTopics
    for topic_id, topicBagOfTopWords in dictOfBagTopWords.items():
        matchingCount = len(bagOfWords & topicBagOfTopWords)        
        topicLikelihoodValue = float(matchingCount)/float(nbTopWords)    
        topiclikelihoodRow[topic_id] = topicLikelihoodValue
    topicLikelihoods.append(topiclikelihoodRow)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/i051796/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
topicLikelihoodFrame.head()

,likelihood_topic_0,likelihood_topic_1,likelihood_topic_2,likelihood_topic_3,likelihood_topic_4,likelihood_topic_5,likelihood_topic_6,likelihood_topic_7,likelihood_topic_8,likelihood_topic_9,...,likelihood_topic_41,likelihood_topic_42,likelihood_topic_43,likelihood_topic_44,likelihood_topic_45,likelihood_topic_46,likelihood_topic_47,likelihood_topic_48,likelihood_topic_49,likelihood_topic_50
0,0.08,0.0,0.00,0.02,0.02,0.02,0.06,0.00,0.0,0.02,...,0.02,0.00,0.0,0.00,0.06,0.00,0.02,0.04,0.02,0.04
1,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,...,0.00,0.02,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0.02,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,...,0.00,0.00,0.0,0.02,0.00,0.00,0.00,0.02,0.00,0.02
3,0.00,0.0,0.02,0.00,0.02,0.00,0.00,0.02,0.0,0.00,...,0.00,0.00,0.0,0.00,0.02,0.02,0.00,0.00,0.00,0.00
4,0.00,0.0,0.00,0.00,0.02,0.00,0.00,0.00,0.0,0.00,...,0.02,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [15]:
names = [ 'likelihood_topic_' + str(i) for i in range(0,51)]
topicLikelihoodFrame = pd.DataFrame(topicLikelihoods, columns = names)
finalXTrain = XTrain.merge(topicLikelihoodFrame, left_index=True, right_index=True)

In [13]:
# save the extraction result
SavedTrain = finalXTrain.drop(["question"], axis=1)
SavedTrain.to_csv("../../data/staging_data/text_extracted_features.csv", index=None)
SavedTrain.head(5)

,ID,drugCount,ingredientCount,timeCount,quantitiesCount,questionMarkCount,sentenceCount,wordCount,combienCount,pourquoiCount,...,likelihoo_topic_41,likelihoo_topic_42,likelihoo_topic_43,likelihoo_topic_44,likelihoo_topic_45,likelihoo_topic_46,likelihoo_topic_47,likelihoo_topic_48,likelihoo_topic_49,likelihoo_topic_50
0,0,0,1,3,0,0,1,74,0,0,...,0.02,0.00,0.0,0.00,0.06,0.00,0.02,0.04,0.02,0.04
1,1,1,0,0,0,0,1,10,0,0,...,0.00,0.02,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2,1,0,0,0,0,1,50,0,0,...,0.00,0.00,0.0,0.02,0.00,0.00,0.00,0.02,0.00,0.02
3,3,1,0,0,0,0,1,14,0,0,...,0.00,0.00,0.0,0.00,0.02,0.02,0.00,0.00,0.00,0.00
4,4,1,1,0,0,0,1,22,0,0,...,0.02,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
